In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import ast
from sklearn.metrics import mean_squared_error, r2_score

# Load data
df = pd.read_csv(r'C:\Apple_leaf\Object_Detection.csv')
images, bboxes = [], []

# Load images and bounding boxes
for index, row in df.iterrows():
    img_array = np.array(ast.literal_eval(row['Image']), dtype=np.uint8)
    if img_array.size == 100 * 100 * 3:
        img_array = img_array.reshape(100, 100, 3)
        images.append(img_array)
        bboxes.append([row['x_min'], row['y_min'], row['x_max'], row['y_max']])
    else:
        print(f"Skipping index {index}: Image size is {img_array.size}, expected {100 * 100 * 3}")

# Convert images and bounding boxes to NumPy arrays
images = np.array(images).reshape(-1, 100, 100, 3)
bboxes = np.array(bboxes, dtype=np.float32)

# Train-test split
X_train, X_test, bbox_train, bbox_test = train_test_split(
    images, bboxes, test_size=0.2, random_state=42
)

# Load a pre-trained model (ResNet50) without the top layers
base_model = tf.keras.applications.ResNet50(
    include_top=False,  # Exclude the fully connected layers
    input_shape=(100, 100, 3),
    weights='imagenet'  # Use ImageNet weights
)

# Freeze the base model layers to prevent them from being updated during training
base_model.trainable = False

# Add custom layers for bounding box prediction
def build_transfer_learning_model(input_shape=(100, 100, 3)):
    input_layer = tf.keras.Input(shape=input_shape)
    x = base_model(input_layer)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    bbox_output = tf.keras.layers.Dense(4, activation='linear')(x)  # Output layer for bounding boxes

    model = tf.keras.Model(inputs=input_layer, outputs=bbox_output, name="Transfer_Learning_Model")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='mean_squared_error'
    )
    return model

# Build the transfer learning model
transfer_model = build_transfer_learning_model()

# Print model summary to verify architecture
transfer_model.summary()

# Train the transfer learning model
transfer_history = transfer_model.fit(
    X_train, bbox_train,
    validation_data=(X_test, bbox_test),
    epochs=25,
    batch_size=32
)

Skipping index 1620: Image size is 40000, expected 30000
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step


Model: "Transfer_Learning_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 4, 4, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │         8,196 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,595,908 (90.01 MB)

 Trainable params: 8,196 (32.02 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

Epoch 1/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 33s 577ms/step - loss: 2910.9888 - val_loss: 1250.7910
Epoch 2/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 559ms/step - loss: 1079.2933 - val_loss: 904.2245
Epoch 3/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 559ms/step - loss: 907.4609 - val_loss: 868.5956
Epoch 4/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 560ms/step - loss: 861.4491 - val_loss: 856.8669
Epoch 5/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 557ms/step - loss: 828.4979 - val_loss: 846.9319
Epoch 6/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 29s 578ms/step - loss: 843.5877 - val_loss: 839.8383
Epoch 7/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 560ms/step - loss: 836.0594 - val_loss: 832.6339
Epoch 8/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 567ms/step - loss: 816.2657 - val_loss: 828.3112
Epoch 9/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 562ms/step - loss: 810.8934 - val_loss: 823.8084
Epoch 10/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 568ms/step - loss: 790.2383 - val_loss: 821.2621
Epoch 11/25
50/50 ━━━━━━━━━━━━━━━━━━━━ 28s 567ms/step - loss: 787.3928 - val_loss: 817

In [ ]:
# IoU Calculation
def calculate_iou(true_bbox, pred_bbox):
    x_min_true, y_min_true, x_max_true, y_max_true = true_bbox
    x_min_pred, y_min_pred, x_max_pred, y_max_pred = pred_bbox
    x_min_inter = max(x_min_true, x_min_pred)
    y_min_inter = max(y_min_true, y_min_pred)
    x_max_inter = min(x_max_true, x_max_pred)
    y_max_inter = min(y_max_true, y_max_pred)
    inter_area = max(0, x_max_inter - x_min_inter) * max(0, y_max_inter - y_min_inter)
    true_area = (x_max_true - x_min_true) * (y_max_true - y_min_true)
    pred_area = (x_max_pred - x_min_pred) * (y_max_pred - y_min_pred)
    union_area = true_area + pred_area - inter_area
    if union_area == 0:
        return 0.0
    iou = inter_area / union_area
    return iou

# Get predictions from the trained model
y_pred_bbox = transfer_model.predict(X_test)

# IoU calculation on predictions
ious = [calculate_iou(true, pred) for true, pred in zip(bbox_test, y_pred_bbox)]
mean_iou = np.mean(ious)
print(f"Mean IoU for bounding box predictions from transfer learning model: {mean_iou*100:.4f}")

# Mean Squared Error and R2 Score
mse = mean_squared_error(bbox_test, y_pred_bbox)
r2 = r2_score(bbox_test, y_pred_bbox)
print(f"Mean Squared Error: {mse*100:.4f}%")
print(f"R2 Score: {r2*100:.4f}%")

# mAP Calculation (Mean Average Precision)
def calculate_map(ious, threshold=0.5):
    tp = sum([1 if iou >= threshold else 0 for iou in ious])
    fp = len(ious) - tp
    fn = 0  # We don't have true negatives in this case, just counting missed detections
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    return precision, recall

# Calculate mAP using IoUs
precision, recall = calculate_map(ious)
map_score = precision * recall  # Simplified mAP calculation, could be extended with averaging
print(f"Mean Average Precision (mAP): {map_score*100:.4f}%")

13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 604ms/step
Mean IoU for bounding box predictions from transfer learning model: 71.3452
Mean Squared Error: 30.6146%
R2 Score: 69.3854%
Mean Average Precision (mAP): 68.9479%
